In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [28]:
# Load the dataset
data = pd.read_csv('/content/tesla.csv')
data = data.dropna()

In [29]:
# Create features and labels
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

<ipython-input-29-6078241aa6e9>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


In [30]:
# Create lagged features
for lag in range(1, 11):
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

In [31]:
# Add more technical indicators as features
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = data['Close'].diff().rolling(window=14).apply(lambda x: np.mean(np.where(x > 0, x, 0)) / (np.mean(np.abs(x)) + 1e-10), raw=True)
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()

In [32]:
data.dropna(inplace=True)

In [33]:
# Prepare feature and target arrays
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'MA10', 'MA50', 'RSI', 'EMA10', 'EMA50'] + [f'Close_lag_{lag}' for lag in range(1, 11)]
X = data[features]
y = data['Close']

In [34]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [36]:
# Initialize the Random Forest Regressor
rf = RandomForestRegressor()

In [37]:
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [38]:
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [39]:
# Get the best estimator
best_rf = grid_search.best_estimator_

In [40]:
# Make predictions on the testing set
y_pred = best_rf.predict(X_test)

In [41]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [42]:
# Calculate a custom accuracy metric
def custom_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs((y_true - y_pred) / y_true) <= tolerance)

accuracy = custom_accuracy(y_test, y_pred)

In [43]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error (MAE): 0.21807545933787478
Mean Squared Error (MSE): 0.36495714514232175
R-squared (R2): 0.9999725378613393
Custom Accuracy: 99.77%


In [44]:
# Make a prediction on new data
new_data = pd.DataFrame({
    'Open': [300],
    'High': [310],
    'Low': [295],
    'Adj Close': [305],
    'Volume': [1000000],
    'MA10': [305],
    'MA50': [310],
    'RSI': [0.01],
    'EMA10': [305],
    'EMA50': [310],
    'Close_lag_1': [298],
    'Close_lag_2': [299],
    'Close_lag_3': [297],
    'Close_lag_4': [296],
    'Close_lag_5': [295],
    'Close_lag_6': [294],
    'Close_lag_7': [293],
    'Close_lag_8': [292],
    'Close_lag_9': [291],
    'Close_lag_10': [290]
})

In [45]:
new_data_scaled = scaler.transform(new_data)
new_prediction = best_rf.predict(new_data_scaled)
print(f'Predicted Close Price: {new_prediction[0]}')

Predicted Close Price: 305.02908989912476
